In [3]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [4]:

# pip install mbutil

In [48]:
# name, areas, centroids, min, max
path = '/Users/danielellis/Desktop/ONS Geography/'

inputs = [
    ['OA'  ,path+'GeoJSON/ons-oa11-bgc.geojson',path+'Raw/Centroids/Output_Areas__December_2011__Population_Weighted_Centroids-shp',1,5],
    ['msoa',path+'GeoJSON/ons-msoa11-bgc.geojson',path+'Raw/Centroids/Middle_Layer_Super_Output_Areas__December_2011__Population_Weighted_Centroids-shp',6,8],
    ['lsoa',path+'GeoJSON/ons-lsoa11-bgc.geojson',path+'Raw/Centroids/Lower_Layer_Super_Output_Areas__December_2011__Population_Weighted_Centroids-shp',9,14]
]



In [50]:
# gpd.read_file(inputs[1][2])
# gen_set(inputs[0])

In [57]:

def gen_set(x):
    '''
    function to generate sets of results in parallel. 
    '''
    
    print(x)
    name, area, centroids, start, end = x
    year = '11'
    cd= 'CD'
    if name != 'OA': 
        cd = 'cd'

    # area     
    base  = gpd.read_file(area)[['areacd','geometry']]
    base.columns = ['id','geometry']
    
    # centroids     
    centroids = gpd.read_file(centroids)[['%s'%(name+year+cd), 'geometry']]
    centroids.columns = ['id','geometry']

    print('save',name)
    base.to_file("tmp/%s.geojson"%(year+name), driver="GeoJSON")
    centroids.to_file("tmp/centroids%s.geojson"%(year+name), driver="GeoJSON")

    print('process',name)

    # cant have centroids being grouped, but fine for area shapes
    os.popen('tippecanoe --read-parallel -Q -Z%d -z%d -f --gamma=0 --no-feature-limit --no-tile-size-limit -r1 -o tiles/centroids%s.mbtiles  ./tmp/centroids%s.geojson'%(start,end,year+name,year+name))
    os.popen('tippecanoe --read-parallel -Q -Z%d -z%d -f -ab -o tiles/outputarea%s.mbtiles --drop-densest-as-needed --generate-id ./tmp/%s.geojson'%(start,end,year,year+name))


    print('join',name)
    os.popen('tile-join -o ./tiles/%s.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/outputarea%s.mbtiles'%(name,year+name,year+name))

    return './tiles/%s.mbtiles'%name

In [ ]:
fs = p_map(gen_set, inputs)

['OA', '/Users/danielellis/Desktop/ONS Geography/GeoJSON/ons-oa11-bgc.geojson', '/Users/danielellis/Desktop/ONS Geography/Raw/Centroids/Output_Areas__December_2011__Population_Weighted_Centroids-shp', 1, 5]['msoa', '/Users/danielellis/Desktop/ONS Geography/GeoJSON/ons-msoa11-bgc.geojson', '/Users/danielellis/Desktop/ONS Geography/Raw/Centroids/Middle_Layer_Super_Output_Areas__December_2011__Population_Weighted_Centroids-shp', 6, 8]['lsoa', '/Users/danielellis/Desktop/ONS Geography/GeoJSON/ons-lsoa11-bgc.geojson', '/Users/danielellis/Desktop/ONS Geography/Raw/Centroids/Lower_Layer_Super_Output_Areas__December_2011__Population_Weighted_Centroids-shp', 9, 14]

  0%|          | 0/3 [00:00<?, ?it/s]




save msoa
process msoa
join msoa


./tiles/centroids11msoa.mbtiles: integrity_check: database is locked
For layer 0, using name "centroids11msoa"
./tmp/centroids11msoa.geojson:454: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:EPSG::27700"}},"features":[]}
./tmp/centroids11msoa.geojson:452: Found ] at top level
For layer 0, using name "11msoa"
7201 features, 136733 bytes of geometry, 16 bytes of separate metadata, 79275 bytes of string pool


More than half the features were clipped away at zoom level 0.
Is your data in the wrong projection? It should be in WGS84/EPSG:4326.
./tmp/11msoa.geojson:315: Found ] at top level
./tmp/11msoa.geojson:802: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
                              
7201 features, 3086448 bytes of geometry, 16 bytes of separate m

save lsoa


process lsoa
join lsoa


./tiles/centroids11lsoa.mbtiles: integrity_check: database is locked
For layer 0, using name "centroids11lsoa"
For layer 0, using name "11lsoa"
./tmp/centroids11lsoa.geojson:2175: Found ] at top level
./tmp/centroids11lsoa.geojson:2175: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:EPSG::27700"}},"features":[]}
34753 features, 683054 bytes of geometry, 16 bytes of separate metadata, 382347 bytes of string pool


More than half the features were clipped away at zoom level 0.
Is your data in the wrong projection? It should be in WGS84/EPSG:4326.

./tmp/11lsoa.geojson:3610: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
./tmp/11lsoa.geojson:1548: Found ] at top level
34753 features, 10286329 bytes of geometry, 27518 bytes of separate metadata, 382347 byt

save OA


In [56]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f %s --attribution=ONS'%' '.join(fs))

./tilesOA.mbtiles: select failed: no such table: tiles


256

In [ ]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf % 2>&1')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf 2>&1')


In [ ]:
os.system("git add -A; git commit -m 'recompute tiles'; git push")